## Logistic Regression
https://towardsdatascience.com/logistic-regression-with-pytorch-3c8bbea594be


In [9]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms

# Define logistic regression model
class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        out = self.linear(x)
        return out

    

# Set hyperparameters
input_dim = 784
output_dim = 10
lr = 0.01
epochs = 10
batch_size = 32
num_clients = 10




# Load MNIST dataset - Normalized with (MEAN - STD)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.MNIST('../data', train=True, download=True, transform=transform)

# Split data among clients randomly using random package random_split
client_datasets = torch.utils.data.random_split(train_dataset, [len(train_dataset) // num_clients] * num_clients)

# Initialize global model // centralized model
global_model = LogisticRegression(input_dim, output_dim)

# Train global model using federated averaging // stochastic gradient descent with cross entropy loss
global_optimizer = optim.SGD(global_model.parameters(), lr=lr)
global_criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
    global_model.train()

    # Train local models on each client
    local_models = []
    for client_dataset in client_datasets:
        local_model = LogisticRegression(input_dim, output_dim)
        
        # Load in the state dict from the global model => Distribute model parameters to clients
        local_model.load_state_dict(global_model.state_dict())
        local_optimizer = optim.SGD(local_model.parameters(), lr=lr)
        local_criterion = nn.CrossEntropyLoss()
        
        
        # Inner training loop to train local models before sending their parameters back for aggregation
        for local_epoch in range(epochs):
            local_model.train()

            for local_data, local_target in torch.utils.data.DataLoader(client_dataset, batch_size=batch_size, shuffle=True):
                local_optimizer.zero_grad()
                local_output = local_model(local_data.view(local_data.shape[0], -1))
                local_loss = local_criterion(local_output, local_target)
                local_loss.backward()
                local_optimizer.step()

        local_models.append(local_model)

    # Update global model using federated averaging
    # Collecting the right parameters
    for name, param in global_model.named_parameters():
        if name.endswith('.bias'):
            continue
        
        # Averaging on the parameters
        local_params = torch.stack([local_model.state_dict()[name] for local_model in local_models])
        global_mean = local_params.mean(0)
        param.data = global_mean.data
    
    # Finding the Global loss and Global accurracy pr. epoch
    global_loss = 0
    global_accuracy = 0
    global_optimizer.zero_grad()

    for global_data, global_target in torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True):
        global_output = global_model(global_data.view(global_data.shape[0], -1))
        global_loss += global_criterion(global_output, global_target)
        global_accuracy += (global_output.argmax(1) == global_target).float().sum()

    global_loss /= len(train_dataset)
    global_accuracy /= len(train_dataset)

    global_loss.backward()
    global_optimizer.step()

    print(f'Epoch {epoch+1} - Global Loss: {global_loss:.4f}, Global Accuracy: {global_accuracy:.4f}')


Epoch 1 - Global Loss: 0.0099, Global Accuracy: 0.9093
Epoch 2 - Global Loss: 0.0092, Global Accuracy: 0.9165
Epoch 3 - Global Loss: 0.0088, Global Accuracy: 0.9199
Epoch 4 - Global Loss: 0.0086, Global Accuracy: 0.9224
Epoch 5 - Global Loss: 0.0085, Global Accuracy: 0.9242
Epoch 6 - Global Loss: 0.0084, Global Accuracy: 0.9251
Epoch 7 - Global Loss: 0.0083, Global Accuracy: 0.9257
Epoch 8 - Global Loss: 0.0082, Global Accuracy: 0.9265
Epoch 9 - Global Loss: 0.0081, Global Accuracy: 0.9272
Epoch 10 - Global Loss: 0.0081, Global Accuracy: 0.9277


## NN 1 Hidden layer - Only real change is the model used (and thus hyperparameters)

In [8]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms

# Define neural network model
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NeuralNetwork, self).__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        return out

    
    
# Set hyperparameters
input_dim = 784
hidden_dim = 128
output_dim = 10
lr = 0.01
epochs = 10
batch_size = 32
num_clients = 10

# Load MNIST dataset - Normalized (MEAN STD)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.MNIST('../data', train=True, download=True, transform=transform)

# Split data among clients
client_datasets = torch.utils.data.random_split(train_dataset, [len(train_dataset) // num_clients] * num_clients)

# Initialize global model
global_model = NeuralNetwork(input_dim, hidden_dim, output_dim)

# Train global model using federated averaging
global_optimizer = optim.SGD(global_model.parameters(), lr=lr)
global_criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
    global_model.train()

    # Train local models on each client
    local_models = []
    for client_dataset in client_datasets:
        local_model = NeuralNetwork(input_dim, hidden_dim, output_dim)
        local_model.load_state_dict(global_model.state_dict())
        local_optimizer = optim.SGD(local_model.parameters(), lr=lr)
        local_criterion = nn.CrossEntropyLoss()

        for local_epoch in range(epochs):
            local_model.train()

            for local_data, local_target in torch.utils.data.DataLoader(client_dataset, batch_size=batch_size, shuffle=True):
                local_optimizer.zero_grad()
                local_output = local_model(local_data.view(local_data.shape[0], -1))
                local_loss = local_criterion(local_output, local_target)
                local_loss.backward()
                local_optimizer.step()

        local_models.append(local_model)

    # Update global model using federated averaging
    for name, param in global_model.named_parameters():
        if name.endswith('.bias'):
            continue

        local_params = torch.stack([local_model.state_dict()[name] for local_model in local_models])
        global_mean = local_params.mean(0)
        param.data = global_mean.data

    global_loss = 0
    global_accuracy = 0
    global_optimizer.zero_grad()

    for global_data, global_target in torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True):
        global_output = global_model(global_data.view(global_data.shape[0], -1))
        global_loss += global_criterion(global_output, global_target)
        global_accuracy += (global_output.argmax(1) == global_target).float().sum()

    global_loss /= len(train_dataset)
    global_accuracy /= len(train_dataset)

    global_loss.backward()
    global_optimizer.step()

    print(f'Epoch {epoch+1} - Global Loss: {global_loss:.4f}, Global Accuracy: {global_accuracy:.4f}')


Epoch 1 - Global Loss: 0.0086, Global Accuracy: 0.9197
Epoch 2 - Global Loss: 0.0066, Global Accuracy: 0.9390
Epoch 3 - Global Loss: 0.0054, Global Accuracy: 0.9503
Epoch 4 - Global Loss: 0.0045, Global Accuracy: 0.9580
Epoch 5 - Global Loss: 0.0039, Global Accuracy: 0.9634
Epoch 6 - Global Loss: 0.0034, Global Accuracy: 0.9678
Epoch 7 - Global Loss: 0.0031, Global Accuracy: 0.9714
Epoch 8 - Global Loss: 0.0028, Global Accuracy: 0.9737
Epoch 9 - Global Loss: 0.0025, Global Accuracy: 0.9757
Epoch 10 - Global Loss: 0.0023, Global Accuracy: 0.9772
